In [1]:
import os

In [2]:
%pwd

'/home/saboa/projects/End-to-End-Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/saboa/projects/End-to-End-Kidney-Disease-Classification'

In [5]:
# creating entity => return type of any function
# written type => return root diroctory
# Update the entity step
from dataclasses import dataclass
from pathlib import Path

# custom entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
# Update the configuration manager in src config
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
# create configuration manager class
# manage entire variables
# Update the configuration manager in src config
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # extracting data ingestion config
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        # store the data ingestion config and store directories
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config


In [9]:
# Update the components
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion():
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the source URL
        '''
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")
        
        except Exception as e:
            raise e
    
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into the data directorty
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Update the Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-16 23:23:24,387: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-16 23:23:24,391: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-16 23:23:24,392: INFO: common: created directory at: artifacts]
[2024-02-16 23:23:24,392: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-16 23:23:24,393: INFO: 2910279481: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=febde83f-298f-4582-8e1d-246d24cd8ef0
To: /home/saboa/projects/End-to-End-Kidney-Disease-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:00<00:00, 70.3MB/s]

[2024-02-16 23:23:25,995: INFO: 2910279481: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]
